In [ ]:
with open("input-example.txt") as f:
    lines = f.readlines()
lines = [l.strip() for l in lines]
lines[:5]

In [283]:
dirs = [(-1, 0), (0, 1), (1, 0), (0, -1)]


def walk(pos, dir):
    return (pos[0] + dir[0], pos[1] + dir[1])


def in_bounds(pos, field):
    return (
        pos[0] >= 0 and pos[0] < len(field) and pos[1] >= 0 and pos[1] < len(field[0])
    )


# Print just one kind of character for debugging
def print_field(c):
    for line in lines:
        for l in line:
            if l == c:
                print(c, end="")
            else:
                print(".", end="")
        print()

In [284]:
# returns amount of plots and fences
def search_perimeter(coord, plant_type, local_visited: set, global_visited: set):
    if coord in local_visited:
        # part of the plot and already been there
        return (0, 0)

    if not in_bounds(coord, lines) or (plant_type != lines[coord[0]][coord[1]]):
        # outward edge -> fence
        return (0, 1)

    global_visited.add(coord)
    local_visited.add(coord)

    perimeter = [
        search_perimeter(walk(coord, d), plant_type, local_visited, global_visited)
        for d in dirs
    ]

    # the coord itself must be added
    plots = 1
    fences = 0
    for p in perimeter:
        plots += p[0]
        fences += p[1]

    return (plots, fences)


# search_perimeter((0,0), "A",set(),set())

In [ ]:
sum = 0
global_visited = set()
for i in range(0, len(lines)):
    for j in range(0, len(lines[0])):
        if (i, j) not in global_visited:
            per = search_perimeter((i, j), lines[i][j], set(), global_visited)
            # print(per)
            sum += per[0] * per[1]

sum

In [286]:
# part 2

In [287]:
# The probably not very optimal idea:
# Get bounding boxes of the area
# Walk around every row and every column through the bounding box for each edge, start one before the bounding box
# We then start edges when we are in an empty field, stop edges when either the adjacent field is empty or the field we are in is full and go line by line doing that
# Case left edge:
# If in an empty field and there is a field to the right, "start" the edge
# The edge get interrupted if either the own field is not empty anymore or the field to the right is empty, so stop in those cases and restart if field to the right is empty again
# Do that in 4 walks for all left, right, top, bottom edges and sum everything up
# Not pretty, but it works

In [ ]:
def get_num_sides(nodes):
    min_y = 99999999
    max_y = -1
    min_x = 99999999
    max_x = -1

    for n in nodes:
        if n[0] < min_y:
            min_y = n[0]
        if n[0] > max_y:
            max_y = n[0]
        if n[1] < min_x:
            min_x = n[1]
        if n[1] > max_x:
            max_x = n[1]
    # print(min_y, max_y, min_x, max_y)

    left_edges = 0
    for j in range(min_x - 1, max_x + 1):
        edge_on = False
        for i in range(min_y, max_y + 1):
            # print(f"Checking: ({i},{j})")
            if (i, j) not in nodes and (i, j + 1) in nodes and not edge_on:
                # print("left edge")
                left_edges += 1
                edge_on = 1
            elif (i, j + 1) not in nodes or (i, j) in nodes:
                edge_on = False

    right_edges = 0
    for j in range(min_x, max_x + 2):
        edge_on = False
        for i in range(min_y, max_y + 1):
            # print(f"Checking: ({i},{j})")
            if (i, j) not in nodes and (i, j - 1) in nodes and not edge_on:
                # print("right edge")
                right_edges += 1
                edge_on = 1
            elif (i, j - 1) not in nodes or (i, j) in nodes:
                edge_on = False

    top_edges = 0
    for i in range(min_y - 1, max_y + 1):
        edge_on = False
        for j in range(min_x, max_x + 1):
            # print(f"Checking: ({i},{j})")
            if (i, j) not in nodes and (i + 1, j) in nodes and not edge_on:
                # print("top edge")
                top_edges += 1
                edge_on = 1
            elif (i + 1, j) not in nodes or (i, j) in nodes:
                edge_on = False

    bot_edges = 0
    for i in range(min_y, max_y + 2):
        edge_on = False
        for j in range(min_x, max_x + 1):
            # print(f"Checking: ({i},{j})")
            if (i, j) not in nodes and (i - 1, j) in nodes and not edge_on:
                # print("bot edge")
                bot_edges += 1
                edge_on = 1
            elif (i - 1, j) not in nodes or (i, j) in nodes:
                edge_on = False

    # print(left_edges)
    # print(right_edges)
    # print(top_edges)
    # print(bot_edges)

    return left_edges + right_edges + top_edges + bot_edges


local_visited = set()
coord = (0, 6)
search_perimeter(coord, lines[coord[0]][coord[1]], local_visited, set())
get_num_sides(local_visited)

In [289]:
# print_field("C")

In [ ]:
sum = 0
global_visited = set()
for i in range(0, len(lines)):
    for j in range(0, len(lines[0])):
        if (i, j) not in global_visited:
            local_visited = set()
            per = search_perimeter((i, j), lines[i][j], local_visited, global_visited)
            # print(local_visited)
            sides = get_num_sides(local_visited)
            # print(f"{lines[i][j]} Nodes: {per[0]} Sides: {sides}")

            sum += len(local_visited) * sides
sum